## 初始运行环境

In [15]:
import os, sys
sys.path.append('..')

from langchain_community.llms.ollama import Ollama
from langchain_huggingface import HuggingFaceEmbeddings
from utils.str_helper import format_documents_vscode

model_name = "llama2-chinese:13b"
llm = Ollama(model=model_name, temperature=0)
model_kwargs = {'device': 'cuda'}
embeddings = HuggingFaceEmbeddings(model_name="../ai_models/m3e-base", model_kwargs=model_kwargs)

### 从巨潮下载股票年报

In [16]:
## 指定股票代码
from utils.cninfo import cninfo

code = '002027'
instance = cninfo(skip_download_stock_json=True)
filter_template = {
    'market': 'szse',  #深沪京
    'tabName': 'fulltext',  #公告
    'plate': [],  #板块
    'category': [],  #公告分类
    'industry': [],  #行业
    'stock': [],  #股票代码
    'searchkey': '',  #标题关键字
    'seDate': '2023-08-01~2024-05-18',  #起始时间
}

# 以10为步长遍历codes
filter_copy = filter_template.copy()
filter_copy['category'] = ['category_ndbg_szsh'] # 年度报告
filter_copy['stock'] = [code]
annoncements = instance.query_announcements_info(filter_copy, download_pdf=True)

data/002027_分众传媒/002027_分众传媒_2023年年度报告摘要_1219912699.pdf
data/002027_分众传媒/002027_分众传媒_2023年年度报告_1219912683.pdf


### 筛选年度报告

In [17]:
for annoncement in annoncements:
    path = annoncement.get("path")
    if "摘要" in path or "英文" in path:
        continue
    else:
        break
path

'data/002027_分众传媒/002027_分众传媒_2023年年度报告_1219912683.pdf'

### 读取指定章节《管理层讨论与分析》

In [18]:
from utils.pdf_helper import read_management_chapter
from langchain_core.documents import Document

page_content = read_management_chapter(path)
metadata = { "source": os.path.basename(path) }
docs = [Document(page_content=page_content, metadata=metadata)]
if os.path.exists("full_page") == False:
    os.mkdir("full_page")
    
md_path = "full_page/" + os.path.basename(path).replace(".pdf", ".md")
with open(md_path, "w") as f:
    f.write(page_content)

### 生成retriever

In [19]:
from langchain.vectorstores import Chroma

from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever, EnsembleRetriever
from utils.CustomMarkdownHeaderTextSplitter import CustomMarkdownHeaderTextSplitter
from utils.KeywordRetriever import KeywordRetriever

DB_PATH = f"vectorstores/db/{code}"
vectorstore = Chroma(persist_directory=DB_PATH, collection_name="split_parents", embedding_function=embeddings)
store = InMemoryStore()

def build_retriever(vectorstore):
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    parent_splitter = CustomMarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on[0:1])
    child_splitter = CustomMarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    parent_doc_retriever = ParentDocumentRetriever(
        vectorstore=vectorstore,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
    )

    keyword_retriever = KeywordRetriever(store=store)
    ensemble_retriever = EnsembleRetriever(retrievers=[parent_doc_retriever, keyword_retriever])
    return ensemble_retriever

ensemble_retriever = build_retriever(vectorstore)
ensemble_retriever.retrievers[0].add_documents(docs)
ensemble_retriever.invoke("公司从事的业务情况")

[Document(page_content='# 三、核心竞争力分析\n## 1、开创了“楼宇电梯”这个户外媒体的核心场景，在主流城市主流人群必经的楼宇电梯空\n间中每天形成高频次的有效到达。\n楼宇电梯是城市的基础设施，楼宇电梯这个最日常的生活场景代表着四个词：主流人群、\n必经、高频、低干扰，而这四个词正是今天引爆品牌的核心资源，由此分众传媒打造了众多\n的品牌引爆经典案例，被评为“中国广告最具品牌引爆力媒体”。\n分众的重要意义在于把品牌渗透到城市主流人群必经的生活场景中，从而实现对用户的\n强效到达。分众已经成为线下流量核心入口，在媒体碎片化的时代将越来越成为品牌引爆的\n核心方式，能够实现对都市主流人群的集中影响。\n## 2、媒体资源规模大、覆盖面广、渗透率高。\n公司通过规模和体量效应所形成的品牌集中引爆能力，持续为客户提供更有效和精准的\n广告投放，从而提升客户对公司媒体价值的认可。作为中国最大的生活圈媒体平台，公司目\n前已覆盖超过 4 亿中国城市主流人群，逐渐发展为贴近消费者生活的核心媒体平台，成为消\n费者生活的重要组成部分。\n## 3、产品差异化布局，数字化、智能化营销能力优势显著。\n凭借覆盖范围广、层次丰富的广告播出网络和管理平台，公司可根据客户不同的投放需\n求，推出不同区域性或媒体平台组合的播放套餐，提供灵活多样、差异化的广告发布策略方\n案。\n近年来，公司持续提升数字化、智能化营销能力，目前已实现云端在线推送，并将绝大\n部分屏幕物联网化，实现了远程在线监控屏幕的播放状态。同时，公司通过对大数据的不断\n深入分析研究及与各平台的合作，根据不同楼宇、社区消费者的不同品类消费需求和品牌偏\n好，持续帮助客户实现精准有效投放，并有效推动品效协同，大幅提升销售转化率，已成为\n重要的线下流量入口。\nAI 技术对广告媒体行业将可能产生革命性的变革，公司积极把握当前 AI 技术所带来的\n机遇，尝试和探索新兴技术的广泛应用。目前，公司营销垂类大模型已完成部署，正持续微\n调和迭代。通过对历史广告数据的深度挖掘，该模型精心构造独有的高质量数据集和合理的\n人工标注反馈数据，并能够在实际场景中持续迭代和优化，持续提升其能力。这样的基于\n12  \n分众传媒信息技术股份有限公司2023年年度报告全文\nAI 驱动的全新行业解决方案，将极大丰

In [20]:
os.startfile(os.path.abspath(path))

In [21]:
bussiness_docs = ensemble_retriever.retrievers[1].invoke("公司从事的主要业务")
if bussiness_docs:
    doc = bussiness_docs[0]
    if os.path.exists("chapter") == False:
        os.mkdir("chapter")
        
    md_path = "chapter/" + os.path.basename(path).replace(".pdf", ".md")
    with open(md_path, "w") as f:
        f.write(doc.page_content)